### Импорт

In [1]:
pip install torchmetrics

     |████████████████████████████████| 329 kB 11.5 MB/s 


In [2]:
pip install ipdb

     |████████████████████████████████| 790 kB 13.1 MB/s 
     |████████████████████████████████| 374 kB 49.9 MB/s 
  Created wheel for ipdb: filename=ipdb-0.13.9-py3-none-any.whl size=11648 sha256=953e024e4be0a7e11eab8e80f04d241494014cedebc842ace808d0a2e069816d
  Stored in directory: /root/.cache/pip/wheels/65/cd/cc/aaf92acae337a28fdd2aa4d632196a59745c8c39f76eaeed01
Successfully built ipdb
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit

In [3]:
import pandas as pd
import numpy as np
from string import punctuation
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
from torchmetrics import F1
from torchmetrics.functional import f1, recall
import ipdb

import gensim

### Готовим данные
https://github.com/daria-sa/NNmethods_ba_hse21-22/blob/main/7_mlp_torch.ipynb

In [4]:
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv 

--2021-11-26 17:27:09--  https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.67.18, 2620:100:6023:18::a27d:4312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.67.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/r6u59ljhhjdg6j0/negative.csv [following]
--2021-11-26 17:27:09--  https://www.dropbox.com/s/raw/r6u59ljhhjdg6j0/negative.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucdeebf682acc8871ff24cdbadd1.dl.dropboxusercontent.com/cd/0/inline/Bat9kgEtQeuxgtP6B8WE69qG1O5V9-6io46Zjc42dBnTMticqZnp_BEajO3AOjraYtOhoGWknHxec_ollG8ewlab73xoZDlC_ouhQ2-hCAiw81xdttpfQduPxpul6PDrECj96pOTRQk4ecxWM_ub4UCO/file# [following]
--2021-11-26 17:27:10--  https://ucdeebf682acc8871ff24cdbadd1.dl.dropboxusercontent.com/cd/0/inline/Bat9kgEtQeuxgtP6B8WE69qG1O5V9-6io46Zjc42dBnTMticqZnp_BEajO3AOjraYtOhoGWknHxec_ollG8ewla

In [5]:
pos_tweets = pd.read_csv('positive.csv', encoding='utf-8', sep=';', header=None,  names=[0,1,2,'text','tone',5,6,7,8,9,10,11])
pos_tweets['tone'] = 1
neg_tweets = pd.read_csv('negative.csv', encoding='utf-8', sep=';', header=None, names=[0,1,2,'text','tone',5,6,7,8,9,10,11])
neg_tweets['tone'] = 0
all_tweets_data = pos_tweets.append(neg_tweets)
print(len(all_tweets_data))

226834


In [6]:
tweets_data = shuffle(all_tweets_data[['text','tone']])
train_sentences, val_sentences = train_test_split(tweets_data, test_size=0.1)

In [7]:
def preprocess(text):
    tokens = text.lower().split()
    tokens = [token.strip(punctuation) for token in tokens]
    return tokens
vocab = Counter()

for text in tweets_data['text']:
    vocab.update(preprocess(text))
print('всего уникальных токенов:', len(vocab))

всего уникальных токенов: 366250


In [8]:
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 2:
        filtered_vocab.add(word)
print('уникальных токенов, втретившихся больше 2 раз:', len(filtered_vocab))

уникальных токенов, втретившихся больше 2 раз: 64423


In [9]:
#создаем словарь с индексами word2id, для спецсимвола паддинга дефолтный индекс - 0
word2id = {'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)
#обратный словарь для того, чтобы раскодировать последовательность
id2word = {i:word for word, i in word2id.items()}

In [10]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [11]:
MAX_LEN = 0

for text in tweets_data.text:
    tokens = preprocess(text)
    MAX_LEN = max(len(tokens), MAX_LEN)
MAX_LEN

40

In [12]:
X = torch.LongTensor(size=(train_sentences.shape[0], MAX_LEN))

for i, text in enumerate(train_sentences.text):
    tokens = preprocess(text) # токенизируем
    
    ids = [word2id[token] for token in tokens if token in word2id][:MAX_LEN]

    ids = F.pad(torch.LongTensor(ids), (0,MAX_LEN-len(ids)))
    X[i] = ids

In [13]:
class TweetsDataset(Dataset):

    def __init__(self, dataset, word2id, DEVICE):
        self.dataset = dataset['text'].values
        self.word2id = word2id
        self.length = dataset.shape[0]
        self.target = dataset['tone'].values
        self.device = DEVICE

    def __len__(self): #это обязательный метод, он должен уметь считать длину датасета
        return self.length

    def __getitem__(self, index): #еще один обязательный метод. По индексу возвращает элемент выборки
        tokens = self.preprocess(self.dataset[index]) # токенизируем
        ids = torch.LongTensor([self.word2id[token] for token in tokens if token in self.word2id])
        y = [self.target[index]]
        return ids, y
    
    def preprocess(self, text):
        tokens = text.lower().split()
        tokens = [token.strip(punctuation) for token in tokens]
        tokens = [token for token in tokens if token]
        return tokens

    def collate_fn(self, batch): #этот метод можно реализовывать и отдельно,
        # он понадобится для DataLoader во время итерации по батчам
        ids, y = list(zip(*batch))
        padded_ids = pad_sequence(ids, batch_first=True).to(self.device)
        #мы хотим применять BCELoss, он будет брать на вход predicted размера batch_size x 1 (так как для каждого семпла модель будет отдавать одно число), target размера batch_size x 1
        y = torch.Tensor(y).to(self.device) # tuple ([1], [0], [1])  -> Tensor [[1.], [0.], [1.]] 
        return padded_ids, y

In [14]:
train_dataset = TweetsDataset(train_sentences, word2id, DEVICE)
train_sampler = SequentialSampler(train_dataset)
train_iterator = DataLoader(train_dataset, collate_fn = train_dataset.collate_fn, sampler=train_sampler, batch_size=1024)

In [15]:
val_dataset = TweetsDataset(val_sentences, word2id, DEVICE)
val_sampler = SequentialSampler(val_dataset)
val_iterator = DataLoader(val_dataset, collate_fn = val_dataset.collate_fn, sampler=val_sampler, batch_size=1024)

### Функции обучения и валидации
https://github.com/daria-sa/NNmethods_ba_hse21-22/blob/main/09_CNN.ipynb

In [16]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0 # для подсчета среднего лосса на всех батчах

    model.train()  # ставим модель в обучение, явно указываем, что сейчас надо будет хранить градиенты у всех весов

    for i, (texts, ys) in enumerate(iterator): #итерируемся по батчам
        optimizer.zero_grad()  #обнуляем градиенты
        preds = model(texts)  #прогоняем данные через модель
        loss = criterion(preds, ys) #считаем значение функции потерь  
        loss.backward() #считаем градиенты  
        optimizer.step() #обновляем веса 
        epoch_loss += loss.item() #сохраняем значение функции потерь
        if not (i + 1) % int(len(iterator)/5):
            print(f'Train loss: {epoch_loss/i}')      
    return  epoch_loss / len(iterator) # возвращаем среднее значение лосса по всей выборке

def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_metric = 0
    model.eval() 
    with torch.no_grad():
        for i, (texts, ys) in enumerate(iterator):   
            preds = model(texts)  # делаем предсказания на тесте
            loss = criterion(preds, ys)   # считаем значения функции ошибки для статистики  
            epoch_loss += loss.item()
            batch_metric = f1(preds.round().long(), ys.long(), ignore_index=0)
            epoch_metric += batch_metric

            if not (i + 1) % int(len(iterator)/5):
                print(f'Val loss: {epoch_loss/i}, Val f1: {epoch_metric/i}')
        
    return epoch_metric / len(iterator), epoch_loss / len(iterator) # возвращаем среднее значение по всей выборке

### Архитектура 1, обучаемые эмбеддинги

In [ ]:
class CNN(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.bigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=100, kernel_size=2, padding='same')
        self.trigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=80, kernel_size=3, padding='same')
        self.bigrams_sec = nn.Conv1d(in_channels=180, out_channels=100, kernel_size=2, padding='same')
        self.hidden = nn.Linear(in_features=100, out_features=1)
        self.out = nn.Sigmoid()

    def forward(self, text):
        embedded = self.embedding(text)
        embedded = embedded.transpose(1,2)
        feature_map_bigrams = self.bigrams(embedded)
        feature_map_trigrams = self.trigrams(embedded)
        concat = torch.cat((feature_map_bigrams, feature_map_trigrams), 1)
        feature_map_bigrams_sec = self.bigrams_sec(concat)
        pooling = feature_map_bigrams_sec.max(2)[0]
        logits = self.hidden(pooling) 
        logits = self.out(logits)      
        return logits

In [ ]:
model = CNN(len(word2id), 100)
optimizer = optim.Adam(model.parameters(), lr=0.0005)
criterion = nn.BCELoss() 

# веса модели и значения лосса храним там же, где и все остальные тензоры
model = model.to(DEVICE)
criterion = criterion.to(DEVICE)

In [ ]:
losses = []
losses_eval = []
f1s = []
f1s_eval = []

for i in range(20):
    print(f'\nstarting Epoch {i}')
    print('Training...')
    epoch_loss = train(model, train_iterator, optimizer, criterion)
    losses.append(epoch_loss)
    print('\nEvaluating on train...')
    f1_on_train,_ = evaluate(model, train_iterator, criterion)
    f1s.append(f1_on_train)
    print('\nEvaluating on test...')
    f1_on_test, epoch_loss_on_test = evaluate(model, val_iterator, criterion)
    losses_eval.append(epoch_loss_on_test)
    f1s_eval.append(f1_on_test)


starting Epoch 0
Training...


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/conv.py:298: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:647.)
  self.padding, self.dilation, self.groups)


Train loss: 0.6847423131649311
Train loss: 0.6571321064912821
Train loss: 0.6436778872954745
Train loss: 0.6339144031956511
Train loss: 0.6264340053850682

Evaluating on train...
Val loss: 0.5983882149060568, Val f1: 0.6943443417549133
Val loss: 0.589506849457946, Val f1: 0.6886432766914368
Val loss: 0.5863617498333714, Val f1: 0.6866886019706726
Val loss: 0.583603135819705, Val f1: 0.6874958872795105
Val loss: 0.5820002337196963, Val f1: 0.6876813173294067

Evaluating on test...
Val loss: 0.7918918132781982, Val f1: 0.8887373805046082
Val loss: 0.6802528074809483, Val f1: 0.7552644610404968
Val loss: 0.6461874571713534, Val f1: 0.721592128276825
Val loss: 0.6311427672704061, Val f1: 0.7099669575691223
Val loss: 0.6208034722428573, Val f1: 0.704522967338562

starting Epoch 1
Training...
Train loss: 0.595395945585691
Train loss: 0.5824286899989164
Train loss: 0.5756451347294975
Train loss: 0.5697135055589976
Train loss: 0.5653553416381529

Evaluating on train...
Val loss: 0.548509565683

### Архитектура 1, предобученные эмбеддинги

In [ ]:
texts = train_sentences.text.apply(preprocess).tolist()
w2v = gensim.models.Word2Vec(texts, size=100, window=5, min_count=1)

weights = np.zeros((len(word2id), 100))
count = 0
for word, i in word2id.items():
    if word == 'PAD':
        continue   
    try:
        weights[i] = w2v.wv[word]    
    except KeyError:
        count += 1
        # oov словам сопоставляем случайный вектор
        weights[i] = np.random.normal(0,0.1,100)

In [ ]:
class CNN_pretrained(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.from_pretrained(torch.tensor(weights), freeze=True)
        self.bigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=100, kernel_size=2, padding='same')
        self.trigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=80, kernel_size=3, padding='same')
        self.bigrams_sec = nn.Conv1d(in_channels=180, out_channels=100, kernel_size=2, padding='same')
        self.hidden = nn.Linear(in_features=100, out_features=1)
        self.out = nn.Sigmoid()

    def forward(self, text):
        embedded = self.embedding(text)
        embedded = embedded.transpose(1,2)
        feature_map_bigrams = self.bigrams(embedded)
        feature_map_trigrams = self.trigrams(embedded)
        concat = torch.cat((feature_map_bigrams, feature_map_trigrams), 1)
        feature_map_bigrams_sec = self.bigrams_sec(concat)
        pooling = feature_map_bigrams_sec.max(2)[0]
        logits = self.hidden(pooling) 
        logits = self.out(logits)      
        return logits

In [ ]:
model = CNN_pretrained(len(word2id), 100)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss() # Binary Cross Entropy

# веса модели и значения лосса храним там же, где и все остальные тензоры
model = model.to(DEVICE)
criterion = criterion.to(DEVICE)

In [ ]:
losses = []
losses_eval = []
f1s = []
f1s_eval = []

for i in range(20):
    print(f'\nstarting Epoch {i}')
    print('Training...')
    epoch_loss = train(model, train_iterator, optimizer, criterion)
    losses.append(epoch_loss)
    print('\nEvaluating on train...')
    f1_on_train,_ = evaluate(model, train_iterator, criterion)
    f1s.append(f1_on_train)
    print('\nEvaluating on test...')
    f1_on_test, epoch_loss_on_test = evaluate(model, val_iterator, criterion)
    losses_eval.append(epoch_loss_on_test)
    f1s_eval.append(f1_on_test)


starting Epoch 0
Training...
Train loss: 0.68256309399238
Train loss: 0.6500966707362404
Train loss: 0.6329495496108752
Train loss: 0.6207773445537256
Train loss: 0.6118407584914011

Evaluating on train...
Val loss: 0.5743533143630395, Val f1: 0.7122806310653687
Val loss: 0.5634657846221441, Val f1: 0.7087022066116333
Val loss: 0.5579672876526328, Val f1: 0.7101188898086548
Val loss: 0.5534333305538826, Val f1: 0.7125400304794312
Val loss: 0.5504873415932583, Val f1: 0.7148285508155823

Evaluating on test...
Val loss: 0.7571129401524862, Val f1: 0.9208321571350098
Val loss: 0.6481303232056754, Val f1: 0.7870772480964661
Val loss: 0.618412044915286, Val f1: 0.7496301531791687
Val loss: 0.6048306902249654, Val f1: 0.7344456315040588
Val loss: 0.5943900315385116, Val f1: 0.7276578545570374

starting Epoch 1
Training...
Train loss: 0.5709325350247897
Train loss: 0.5543369575391842
Train loss: 0.5449933013996157
Train loss: 0.5377452420363636
Train loss: 0.5330143569402359

Evaluating on t

Из лоссов видно, что в целом модель достаточно сложная, чтобы выучить закономерности в данных: она показывает хорошие результаты на трейне. Однако при валидации на тестовой выборке результаты значительно ниже, что свидетельствует о том, что модель переобучается. Например, на предобученных эмбеддингах она делает это уже после второй эпохи: лоссы на тесте начинают расти обратно. При этом модель в целом быстрее обучается на предобученных эмбеддингах, поэтому модифицировать будем эту версию.

### Архитектура 1, предобученные эмбеддинги + модификация

In [ ]:
texts = train_sentences.text.apply(preprocess).tolist()
w2v = gensim.models.Word2Vec(texts, size=300, window=5, min_count=1)

weights = np.zeros((len(word2id), 300))
count = 0
for word, i in word2id.items():
    if word == 'PAD':
        continue   
    try:
        weights[i] = w2v.wv[word]    
    except KeyError:
        count += 1
        # oov словам сопоставляем случайный вектор
        weights[i] = np.random.normal(0,0.1,300)

In [ ]:
class CNN_pretrained_mod(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.from_pretrained(torch.tensor(weights), freeze=True)
        self.bigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=100, kernel_size=2, padding='same')
        self.trigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=80, kernel_size=3, padding='same')
        self.sec = nn.Conv1d(in_channels=180, out_channels=180, kernel_size=4, padding='same')
        self.third = nn.Conv1d(in_channels=180, out_channels=100, kernel_size=3, padding='same')
        self.dropout = nn.Dropout(p=0.5)
        self.relu = nn.ReLU()
        self.hidden = nn.Linear(in_features=100, out_features=1)
        self.out = nn.Sigmoid()

    def forward(self, text):
        embedded = self.embedding(text)
        embedded = embedded.transpose(1,2)
        feature_map_bigrams = self.dropout(self.relu(self.bigrams(embedded)))
        feature_map_trigrams = self.dropout(self.relu(self.trigrams(embedded)))
        concat = torch.cat((feature_map_bigrams, feature_map_trigrams), 1)
        feature_map_sec = self.sec(concat)
        feature_map_third = self.third(feature_map_sec)
        pooling = feature_map_third.max(2)[0]
        logits = self.hidden(pooling) 
        logits = self.out(logits)      
        return logits

In [ ]:
model = CNN_pretrained_mod(len(word2id), 300)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss() # Binary Cross Entropy

# веса модели и значения лосса храним там же, где и все остальные тензоры
model = model.to(DEVICE)
criterion = criterion.to(DEVICE)

In [ ]:
losses = []
losses_eval = []
f1s = []
f1s_eval = []

for i in range(10):
    print(f'\nstarting Epoch {i}')
    print('Training...')
    epoch_loss = train(model, train_iterator, optimizer, criterion)
    losses.append(epoch_loss)
    print('\nEvaluating on train...')
    f1_on_train,_ = evaluate(model, train_iterator, criterion)
    f1s.append(f1_on_train)
    print('\nEvaluating on test...')
    f1_on_test, epoch_loss_on_test = evaluate(model, val_iterator, criterion)
    losses_eval.append(epoch_loss_on_test)
    f1s_eval.append(f1_on_test)


starting Epoch 0
Training...
Train loss: 0.683569229566134
Train loss: 0.649019021776658
Train loss: 0.6299428994916066
Train loss: 0.6166419146945642
Train loss: 0.6060836938158352

Evaluating on train...
Val loss: 0.5484773761186844, Val f1: 0.739396870136261
Val loss: 0.5383420126347602, Val f1: 0.7331531047821045
Val loss: 0.531888597402252, Val f1: 0.7340630888938904
Val loss: 0.5279921694371685, Val f1: 0.7361087799072266
Val loss: 0.5255648886139069, Val f1: 0.7374288439750671

Evaluating on test...
Val loss: 0.7161086996396383, Val f1: 0.9595696926116943
Val loss: 0.62043354340962, Val f1: 0.8167509436607361
Val loss: 0.5917400013316761, Val f1: 0.777010440826416
Val loss: 0.5780653119087219, Val f1: 0.7594258785247803
Val loss: 0.5709833659623799, Val f1: 0.7496312260627747

starting Epoch 1
Training...
Train loss: 0.5536852601246957
Train loss: 0.5379985171028331
Train loss: 0.5278169137089193
Train loss: 0.5209205146855528
Train loss: 0.5150326809391903

Evaluating on train

С дропаутами стало действительно лучший лосс на тесте действительно был немного получше, но буквально на одну сотую. Зато скетка медленнее переобучается. Если увличить размер эмбеддинга до 300, становится еще немного лучше. Плюс я добавила еще одну свертку после конкатенации.
Можно сравнить лучший лосс на тесте у модели со всеми модификациями и лучший лосс на тесте у CNN_pretrained. 

CNN_pretrained

    Val loss: 0.6978476444880167, Val f1: 0.97179114818573
    Val loss: 0.5950382266725812, Val f1: 0.8298261165618896
    Val loss: 0.5670244639570062, Val f1: 0.7905034422874451
    Val loss: 0.5577905774116516, Val f1: 0.7714343667030334
    Val loss: 0.5465866139060572, Val f1: 0.7661719918251038

CNN_pretrained_mod 

    Val loss: 0.6252497831980387, Val f1: 1.0403616428375244
    Val loss: 0.5483785441943577, Val f1: 0.8879547715187073
    Val loss: 0.5217799761078574, Val f1: 0.8472750782966614
    Val loss: 0.5141767422358196, Val f1: 0.8258144855499268
    Val loss: 0.5075135058478305, Val f1: 0.81560218334198

### Готовим данные 2

In [17]:
vocab = Counter()
for text in tweets_data['text']:
    vocab.update(list(text))
print('всего уникальных символов:', len(vocab))

всего уникальных символов: 456


In [18]:
filtered_vocab = set()

for symbol in vocab:
    if vocab[symbol] > 5:
        filtered_vocab.add(symbol)
print('уникальных символов, втретившихся больше 5 раз:', len(filtered_vocab))

уникальных символов, втретившихся больше 5 раз: 234


In [19]:
#создаем словарь с индексами symbol2id, для спецсимвола паддинга дефолтный индекс - 0
symbol2id = {'PAD':0}

for symbol in filtered_vocab:
    symbol2id[symbol] = len(symbol2id)

#обратный словарь для того, чтобы раскодировать последовательность
id2symbol = {i:symbol for symbol, i in symbol2id.items()}
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [20]:
class TweetsDataset2(Dataset):

    def __init__(self, dataset, symbol2id, word2id, DEVICE):
        self.dataset = dataset['text'].values
        self.symbol2id = symbol2id
        self.word2id = word2id
        self.length = dataset.shape[0]
        self.target = dataset['tone'].values
        self.device = DEVICE

    def __len__(self): #это обязательный метод, он должен уметь считать длину датасета
        return self.length

    def __getitem__(self, index): #еще один обязательный метод. По индексу возвращает элемент выборки
        text = self.dataset[index]
        tokens = self.preprocess(text) # токенизируем
        ids = torch.LongTensor([self.word2id[token] for token in tokens if token in self.word2id])
        symbol_ids = torch.LongTensor([self.symbol2id[symbol] for symbol in text if symbol in self.symbol2id])
        y = [self.target[index]]
        return ids, symbol_ids, y
    
    def preprocess(self, text):
        tokens = text.lower().split()
        tokens = [token.strip(punctuation) for token in tokens]
        tokens = [token for token in tokens if token]
        return tokens

    def collate_fn(self, batch): #этот метод можно реализовывать и отдельно,
    # он понадобится для DataLoader во время итерации по батчам
      ids, symbol_ids, y = list(zip(*batch))
      padded_symbol_ids = pad_sequence(symbol_ids, batch_first=True).to(self.device)
      padded_ids = pad_sequence(ids, batch_first=True).to(self.device)
      #мы хотим применять BCELoss, он будет брать на вход predicted размера batch_size x 1 (так как для каждого семпла модель будет отдавать одно число), target размера batch_size x 1
      y = torch.Tensor(y).to(self.device) # tuple ([1], [0], [1])  -> Tensor [[1.], [0.], [1.]] 
      return (padded_ids, padded_symbol_ids), y

In [21]:
train_dataset = TweetsDataset2(train_sentences, symbol2id, word2id, DEVICE)
train_sampler = RandomSampler(train_dataset)
train_iterator = DataLoader(train_dataset, collate_fn = train_dataset.collate_fn, sampler=train_sampler, batch_size=1024)

In [22]:
val_dataset = TweetsDataset2(val_sentences, symbol2id, word2id, DEVICE)
val_sampler = RandomSampler(val_dataset)
val_iterator = DataLoader(val_dataset, collate_fn = val_dataset.collate_fn, sampler=val_sampler, batch_size=1024)

### Архитектура 2

In [23]:
class CNN2(nn.Module):
    
    def __init__(self, vocab_size_word, vocab_size_symbol, embedding_dim):
        super().__init__()
        self.embedding_word = nn.Embedding(vocab_size_word, embedding_dim)
        self.hidden_word = nn.Linear(in_features=embedding_dim, out_features=100)
        self.embedding_symbol = nn.Embedding(vocab_size_symbol, embedding_dim)
        self.bigrams_symbol = nn.Conv1d(in_channels=embedding_dim, out_channels=100, kernel_size=2, padding='same')
        self.trigrams_symbol = nn.Conv1d(in_channels=embedding_dim, out_channels=80, kernel_size=3, padding='same')
        self.hidden = nn.Linear(in_features=280, out_features=1)
        self.out = nn.Sigmoid()

    def forward(self, text):
        embedded_word = self.embedding_word(text[0])
        embedded_sent = torch.mean(embedded_word, dim=1)
        embedded_sent = self.hidden_word(embedded_sent)

        embedded_symbol = self.embedding_symbol(text[1])
        embedded_symbol = embedded_symbol.transpose(1,2)
        feature_map_bigrams = self.bigrams_symbol(embedded_symbol)
        feature_map_trigrams = self.trigrams_symbol(embedded_symbol)
        concat = torch.cat((feature_map_bigrams, feature_map_trigrams), 1)
        pooling = concat.max(2)[0]

        concat = torch.cat((embedded_sent, pooling), 1)
        logits = self.hidden(concat) 
        logits = self.out(logits)      
        return logits

In [24]:
model = CNN2(len(word2id), len(symbol2id), 100)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss() # Binary Cross Entropy

# веса модели и значения лосса храним там же, где и все остальные тензоры
model = model.to(DEVICE)
criterion = criterion.to(DEVICE)

In [25]:
losses = []
losses_eval = []
f1s = []
f1s_eval = []

for i in range(10):
    print(f'\nstarting Epoch {i}')
    print('Training...')
    epoch_loss = train(model, train_iterator, optimizer, criterion)
    losses.append(epoch_loss)
    print('\nEvaluating on train...')
    f1_on_train,_ = evaluate(model, train_iterator, criterion)
    f1s.append(f1_on_train)
    print('\nEvaluating on test...')
    f1_on_test, epoch_loss_on_test = evaluate(model, val_iterator, criterion)
    losses_eval.append(epoch_loss_on_test)
    f1s_eval.append(f1_on_test)


starting Epoch 0
Training...


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/conv.py:298: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:647.)
  self.padding, self.dilation, self.groups)


Train loss: 0.3490513395040463
Train loss: 0.18837691278821683
Train loss: 0.12850420717776073
Train loss: 0.09754185938077199
Train loss: 0.07857494403891284

Evaluating on train...
Val loss: 0.0023157291848642323, Val f1: 1.0255930423736572
Val loss: 0.0024024410205716385, Val f1: 1.0125236511230469
Val loss: 0.002356087035663864, Val f1: 1.0082893371582031
Val loss: 0.002359549527396243, Val f1: 1.0061793327331543
Val loss: 0.0023539833245967427, Val f1: 1.0049028396606445

Evaluating on test...
Val loss: 0.0035546041714648404, Val f1: 1.332688570022583
Val loss: 0.003054564253294042, Val f1: 1.1423078775405884
Val loss: 0.00288235245865177, Val f1: 1.0905596017837524
Val loss: 0.0030178340928008157, Val f1: 1.0662821531295776
Val loss: 0.0028176211604946538, Val f1: 1.0523279905319214

starting Epoch 1
Training...
Train loss: 0.0020871655149862934
Train loss: 0.0018177537086914895
Train loss: 0.001598788721745192
Train loss: 0.0014294062689116588
Train loss: 0.0012837122537426174

